In [44]:
import csv, re
import pandas as pd

csv_name = 'dupes'

df = pd.read_csv(csv_name + '.csv')

In [45]:

plates = df['PlateName'].drop_duplicates().values

hasDupes = False
warning_message = []

message = ''
for plate in plates:
    batch = df.where(df['PlateName'] == plate).dropna()
    
    dupes_mask = batch.duplicated(subset = 'SourceWell')
    dupes = batch.where(dupes_mask).dropna()
    duplicates = set(dupes['SourceWell'].values)
    
    if len(duplicates) == 0:
        continue
    else:
        hasDupes = True
    
    message = 'Plate %s has duplicates: \n ' % plate
    for element in duplicates:
        subset = batch.where(batch['SourceWell'] == element).dropna()
        
        indices = []
        for index, row in subset.iterrows():
            indices.append(index)
        message = message + 'SourceWell %s is duplicated in rows %s' % (element, indices) + '\n'
    
    warning_message.append(message)
    
for msg in warning_message:
    print(msg)

Plate p2 has duplicates: 
 SourceWell A2 is duplicated in rows [5, 6]

Plate p3 has duplicates: 
 SourceWell B5 is duplicated in rows [9, 10]
SourceWell H1 is duplicated in rows [11, 12, 13]



In [34]:
df.columns

Index(['PlateName', 'SourceWell', 'TargetWell'], dtype='object')

In [48]:
class Transfer:
    '''
    Stores a list of plates with source-dest pairs
    '''
    def __init__(self, name):
        self.df = pd.read_csv(name)
    
    def checkDuplicateDestination(self):
        hasDupes = False
        dupes_mask = self.df.duplicated(subset = 'TargetWell')
        dupes = self.df.where(dupes_mask).dropna()
        duplicates = set(dupes['TargetWell'].values)
        
        if len(duplicates)==0:
            return hasDupes
        else:
            hasDupes = True
        
        for element in duplicates:
            subset = self.df.where(df['TargetWell'] == element).dropna()

            indices = []
            for index, row in subset.iterrows():
                indices.append(index)
                target = row['TargetWell']
            print('TargetWell %s is duplicated in rows %s' % (target, indices))
        return hasDupes

    def checkDuplicateSource(self):
        plates = self.df['PlateName'].drop_duplicates().values

        hasDupes = False
        warning_message = []

        message = ''
        for plate in plates:
            batch = self.df.where(df['PlateName'] == plate).dropna()

            dupes_mask = batch.duplicated(subset = 'SourceWell')
            dupes = batch.where(dupes_mask).dropna()
            duplicates = set(dupes['SourceWell'].values)

            if len(duplicates) == 0:
                continue
            else:
                hasDupes = True

            message = 'Plate %s has duplicates: \n ' % plate
            for element in duplicates:
                subset = batch.where(batch['SourceWell'] == element).dropna()

                indices = []
                for index, row in subset.iterrows():
                    indices.append(index)
                message = message + 'SourceWell %s is duplicated in rows %s' % (element, indices) + '\n'

            warning_message.append(message)

        for msg in warning_message:
            print(msg)
            
        return hasDupes


In [54]:
well_rows = [chr(x) for x in range(ord('A'), ord('H') + 1)] # move to state machine
# make a list of well names in column wise order 
well_names = []
for i in range(1, 13):
    for letter in well_rows:
        well_names.append(letter+str(i))
        
well_names

['A1',
 'B1',
 'C1',
 'D1',
 'E1',
 'F1',
 'G1',
 'H1',
 'A2',
 'B2',
 'C2',
 'D2',
 'E2',
 'F2',
 'G2',
 'H2',
 'A3',
 'B3',
 'C3',
 'D3',
 'E3',
 'F3',
 'G3',
 'H3',
 'A4',
 'B4',
 'C4',
 'D4',
 'E4',
 'F4',
 'G4',
 'H4',
 'A5',
 'B5',
 'C5',
 'D5',
 'E5',
 'F5',
 'G5',
 'H5',
 'A6',
 'B6',
 'C6',
 'D6',
 'E6',
 'F6',
 'G6',
 'H6',
 'A7',
 'B7',
 'C7',
 'D7',
 'E7',
 'F7',
 'G7',
 'H7',
 'A8',
 'B8',
 'C8',
 'D8',
 'E8',
 'F8',
 'G8',
 'H8',
 'A9',
 'B9',
 'C9',
 'D9',
 'E9',
 'F9',
 'G9',
 'H9',
 'A10',
 'B10',
 'C10',
 'D10',
 'E10',
 'F10',
 'G10',
 'H10',
 'A11',
 'B11',
 'C11',
 'D11',
 'E11',
 'F11',
 'G11',
 'H11',
 'A12',
 'B12',
 'C12',
 'D12',
 'E12',
 'F12',
 'G12',
 'H12']

In [49]:
t = Transfer('dupes.csv')
t.checkDuplicateDestination()
t.checkDuplicateSource()

TargetWell A2 is duplicated in rows [0, 1, 4]
TargetWell D3 is duplicated in rows [8, 9]
Plate p2 has duplicates: 
 SourceWell A2 is duplicated in rows [5, 6]

Plate p3 has duplicates: 
 SourceWell B5 is duplicated in rows [9, 10]
SourceWell H1 is duplicated in rows [11, 12, 13]



True

In [9]:
p = [1, 2 ,3]
d = [2, 3]

a = zip(p,d)

AttributeError: 'list' object has no attribute 'unique'

In [16]:
1 in p

True